In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import randint
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

In [13]:
def scale(x_train, x_test):
    scaler = StandardScaler()
    scaler.fit(x_train)

    x_train = pd.DataFrame(scaler.transform(x_train), 
                           index=x_train.index, 
                           columns=x_train.columns)

    x_test = pd.DataFrame(scaler.transform(x_test),
                          index=x_test.index, 
                          columns=x_test.columns)
    return x_train, x_test

In [14]:
x = pd.read_csv('processed_data/x_post_preproc.csv', index_col='building_id')
y = pd.read_csv('processed_data/y_post_preproc.csv', index_col='building_id')

In [15]:
x_train, x_test = train_test_split(x, test_size=0.2, random_state=42)
y_train, y_test = y.loc[x_train.index], y.loc[x_test.index]

In [16]:
x_train_norm, x_test_norm = scale(x_train, x_test)

## 1. Random Forests: 
    ensemble method bagging decision trees (default bootstrapping = True is kept)

In [22]:
pipeline = make_pipeline(StandardScaler(), 
                         RandomForestClassifier(verbose=1))

In [23]:
param_distribs = {'randomforestclassifier__n_estimators': [50, 100, 200], 
                  'randomforestclassifier__max_features': [5, 10, 12],
                  'randomforestclassifier__max_depth': [5, 10, 20]}

In [24]:
grid_search = GridSearchCV(pipeline, param_distribs, cv=2, verbose=1)

In [25]:
grid_search.fit(x_train_norm, y_train.values.ravel())

Fitting 2 folds for each of 27 candidates, totalling 54 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    7.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    7.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(verbose=1))]),
             param_grid={'randomforestclassifier__max_depth': [5, 10, 20],
                         'randomforestclassifier__max_features': [5, 10, 12],
                         'randomforestclassifier__n_estimators': [50, 100,
                                                                  200]},
             verbose=1)

In [26]:
grid_search.best_estimator_

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=20, max_features=12,
                                        n_estimators=200, verbose=1))])

In [27]:
grid_search.best_score_

0.5907983754888302

In [28]:
joblib.dump(grid_search, 'results/rf_gridsearch.joblib', compress=1)

['results/rf_gridsearch.joblib']

## 2. Boosting: 
    ensemble method boosting decision trees

In [23]:
fit_params = {'early_stopping_rounds': 10,
              'eval_metric': 'merror',
              'eval_set': [(x_test_norm, y_test.values.ravel())]}

param_distribs = {'n_estimators': [50, 100, 200], 
                  'max_depth': [5, 10, 20],
                  'learning_rate': [0.08, 0.15, 0.35]}

In [26]:
grid_search = GridSearchCV(XGBClassifier(), param_distribs, verbose=1, cv=2)
grid_search.fit(x_train_norm, y_train.values.ravel(), **fit_params)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-merror:0.45388
[1]	validation_0-merror:0.45267
[2]	validation_0-merror:0.45040
[3]	validation_0-merror:0.45015
[4]	validation_0-merror:0.44964
[5]	validation_0-merror:0.44985
[6]	validation_0-merror:0.44965
[7]	validation_0-merror:0.44936
[8]	validation_0-merror:0.44905
[9]	validation_0-merror:0.44927
[10]	validation_0-merror:0.44891
[11]	validation_0-merror:0.44886
[12]	validation_0-merror:0.44868
[13]	validation_0-merror:0.44850
[14]	validation_0-merror:0.44801
[15]	validation_0-merror:0.44775
[16]	validation_0-merror:0.44755
[17]	validation_0-merror:0.44707
[18]	validation_0-merror:0.44695
[19]	validation_0-merror:0.44678
[20]	validation_0-merror:0.44643
[21]	validation_0-merror:0.44634
[22]	validation_0-merror:0.44609
[23]	validation_0-merror:0.44597
[24]	validation_0-merror:0.44575
[25]	validation_0-merror:0.44573
[26]	validation_0-merror:0.44551
[27]	validation_0-merror:0.44531
[28]	validation_0-merror:0.44528
[29]	validation_0-merror:0.44504
[30]	validation_0-me

C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-merror:0.45313
[1]	validation_0-merror:0.45149
[2]	validation_0-merror:0.45063
[3]	validation_0-merror:0.45071
[4]	validation_0-merror:0.45032
[5]	validation_0-merror:0.44991
[6]	validation_0-merror:0.45021
[7]	validation_0-merror:0.45022
[8]	validation_0-merror:0.44978
[9]	validation_0-merror:0.44918
[10]	validation_0-merror:0.44872
[11]	validation_0-merror:0.44853
[12]	validation_0-merror:0.44856
[13]	validation_0-merror:0.44822
[14]	validation_0-merror:0.44784
[15]	validation_0-merror:0.44772
[16]	validation_0-merror:0.44735
[17]	validation_0-merror:0.44737
[18]	validation_0-merror:0.44665
[19]	validation_0-merror:0.44662
[20]	validation_0-merror:0.44660
[21]	validation_0-merror:0.44630
[22]	validation_0-merror:0.44586
[23]	validation_0-merror:0.44567
[24]	validation_0-merror:0.44542
[25]	validation_0-merror:0.44532
[26]	validation_0-merror:0.44517
[27]	validation_0-merror:0.44505
[28]	validation_0-merror:0.44462
[29]	validation_0-merror:0.44445
[30]	validation_0-me

...

C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-merror:0.44740
[1]	validation_0-merror:0.43531
[2]	validation_0-merror:0.43107
[3]	validation_0-merror:0.42880
[4]	validation_0-merror:0.42718
[5]	validation_0-merror:0.42627
[6]	validation_0-merror:0.42543
[7]	validation_0-merror:0.42539
[8]	validation_0-merror:0.42490
[9]	validation_0-merror:0.42494
[10]	validation_0-merror:0.42508
[11]	validation_0-merror:0.42507
[12]	validation_0-merror:0.42508
[13]	validation_0-merror:0.42535
[14]	validation_0-merror:0.42575
[15]	validation_0-merror:0.42618
[16]	validation_0-merror:0.42605
[17]	validation_0-merror:0.42617
[18]	validation_0-merror:0.42627


C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-merror:0.43857
[1]	validation_0-merror:0.43445
[2]	validation_0-merror:0.43233
[3]	validation_0-merror:0.43151
[4]	validation_0-merror:0.42985
[5]	validation_0-merror:0.42957
[6]	validation_0-merror:0.42960
[7]	validation_0-merror:0.42902
[8]	validation_0-merror:0.42832
[9]	validation_0-merror:0.42804
[10]	validation_0-merror:0.42778
[11]	validation_0-merror:0.42783
[12]	validation_0-merror:0.42730
[13]	validation_0-merror:0.42699
[14]	validation_0-merror:0.42657
[15]	validation_0-merror:0.42596
[16]	validation_0-merror:0.42568
[17]	validation_0-merror:0.42558
[18]	validation_0-merror:0.42529
[19]	validation_0-merror:0.42466
[20]	validation_0-merror:0.42451
[21]	validation_0-merror:0.42413
[22]	validation_0-merror:0.42348
[23]	validation_0-merror:0.42301
[24]	validation_0-merror:0.42285
[25]	validation_0-merror:0.42259
[26]	validation_0-merror:0.42240
[27]	validation_0-merror:0.42200
[28]	validation_0-merror:0.42196
[29]	validation_0-merror:0.42155
[30]	validation_0-me

GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None,

In [27]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.15, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
joblib.dump(grid_search, 'results/xgb_gridsearch.joblib', compress=1)

['results/xgb_gridsearch.joblib']

## 3. Logistic model:

In [28]:
params = {'C' : np.logspace(-3, 3, 3),
          'penalty': ['l1', 'l2', 'elasticnet']}

logreg=LogisticRegression(solver='saga', multi_class='multinomial', verbose=1, l1_ratio=0.5, max_iter=200)

In [29]:
grid_search=GridSearchCV(logreg, params, cv=2)
grid_search.fit(x_train_norm, y_train.values.ravel())

C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 27 epochs took 37 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.9s finished
C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 28 epochs took 41 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   40.9s finished
C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

convergence after 125 epochs took 205 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min finished
C:\Users\aleciu\Anaconda3\envs\ml\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 107 epochs took 199 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.3min finished


GridSearchCV(cv=2,
             estimator=LogisticRegression(l1_ratio=0.5, max_iter=200,
                                          multi_class='multinomial',
                                          solver='saga', verbose=1),
             param_grid={'C': array([1.e-03, 1.e+00, 1.e+03]),
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [32]:
grid_search.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [31]:
grid_search.best_score_

0.5508607038348818

In [41]:
joblib.dump(grid_search, 'results/log_gridsearch.joblib', compress=1)

['results/log_gridsearch.joblib']